# Logistic Regression

### Data Loading

In [42]:
import pandas as pd
import numpy as np

trn_imputed_dum_X = pd.read_csv('./Dataset/trn_imputed_dum_X.csv')
tst_imputed_dum_X = pd.read_csv('./Dataset/tst_imputed_dum_X.csv')
trn_origin_Y      = pd.read_csv('./Dataset/trn_origin_Y.csv')
tst_origin_X      = pd.read_csv('./Dataset/tst_origin_X.csv')

print(trn_imputed_dum_X.shape)
print(trn_origin_Y.shape)

(79853, 17)
(79853, 1)


### 1) Hyperparameter optimization

In [43]:
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# Parameters
param = {'C':[0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1],
           'tol' : [1e-15, 1e-14, 1e-13, 1e-12, 1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}

param = {'C':[0.001, 0.005],
           'tol' : [1e-15, 1e-14]}

def clf_LogisticLR(trn_X, trn_Y, lr_param):
    clf_LR = linear_model.LogisticRegression()
    clf = GridSearchCV(clf_LR, lr_param, n_jobs=8, cv=10, scoring='roc_auc')
    clf.fit(trn_X, trn_Y)
    return clf

### 2) Training Model

In [40]:
clf_model = clf_LogisticLR(trn_imputed_dum_X, trn_origin_Y.values.ravel(), param)


### 3) DO PREDICTION

In [44]:
clf_yhat = clf_model.best_estimator_.predict(tst_imputed_dum_X)
clf_prob = clf_model.best_estimator_.predict_proba(tst_imputed_dum_X)
clf_prob_renewal = clf_prob[:,1]
display(clf_prob_renewal)

array([0.97973469, 0.97589855, 0.92588906, ..., 0.9516387 , 0.80682406,
       0.96422339])

### 4) Export to output file

In [46]:
tst_output = pd.DataFrame(tst_origin_X[['id', 'premium']])

clf_prob_df = pd.DataFrame(clf_prob_renewal.reshape(len(clf_prob_renewal),1), columns=['renewal'])
pd_incentives = pd.DataFrame(np.zeros((len(clf_prob_renewal), 1)), columns=['incentives'], dtype=np.float32)
pd_improve = pd.DataFrame(np.zeros((len(clf_prob_renewal), 1)), columns=['improvement'], dtype=np.float32)

tst_output = pd.concat([tst_outfrom sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFoldput, clf_prob_df, pd_improve, pd_incentives], axis=1)
#display(tst_output)

tst_output.to_csv('./Dataset/tst_renewal_predicted_LR.csv', index=False)